# Step 2: Looking at the Data Level 2


In this notebook we will analyze Crab DL2 collected with MAGIC+LST1 in several nights scattered over 2020 and 2021. We will produce ($\theta^2$) plots, will check the achieved angular resolution and will produce sky map. 
We will start, however, by taking a closer look into the data itself.


### Let's start by loading some useful modules

In [ ]:
import glob
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
from pyirf.benchmarks import angular_resolution
from astropy import units as u
from astropy.table import Table, QTable
from astropy.coordinates import SkyCoord
from gammapy.stats import WStatCountsStatistic
from magicctapipe.io import get_dl2_mean, get_stereo_events
from magicctapipe.utils import calculate_off_coordinates
from matplotlib import pyplot as plt

In [ ]:
# Display all the columns of a pandas data frame
pd.set_option("display.max_columns", None)

# Customize the pyplot figure
plt.rcParams.update(
    {"figure.figsize": (12, 9), "font.size": 15, "grid.linestyle": "dotted"}
)

# Get the pyplot default color cycle
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

### Load input DL2 data files (Crab observations)

In this example we will explore the data from the directory /fefs/aswg/workspace/2023_joint_analysis_school /RFs_and_DL2/input/input_step_2/DL2-real/ 
Let's start by listing the input files with _glob_:

In [ ]:
#input_file_mask = '/fefs/aswg/workspace/2023_joint_analysis_school/RFs_and_DL2/input/input_step_2/DL2-real/*.h5'

#larger statistics may create kernel problems...
#input_file_mask = '/home/dipierr/data/software-school-2023/RFs_and_DL2/input/input_step_2/DL2-real/*.h5'

#reduced statistics in case of kernel problems
input_file_mask = '/home/dipierr/data/software-school-2023/RFs_and_DL2/input/input_step_2/DL2-real/*Run03*.h5'

# Find the input files
input_files = glob.glob(input_file_mask)
input_files.sort()

Now let's read all of the data files and concatenate them in a single data frame `event_data`:

In [ ]:
print("The following files are found:")

data_list = []

for input_file in input_files:

    print(Path(input_file).name)

    # Load the input file
    df_events = pd.read_hdf(input_file, key="events/parameters")
    data_list.append(df_events)

event_data = pd.concat(data_list)
event_data.set_index(["obs_id", "event_id", "tel_id"], inplace=True)
event_data.sort_index(inplace=True)

### Applying cuts
Now let's apply a quality cut to the data packed in the table "event_data" and take a look at it. We see that most of the events are detected by all three telescopes. Note that the "events" here are real data (before any gammaness cut).

In [ ]:
quality_cuts = f"(disp_diff_mean < {np.sqrt(0.05)})"

# Apply the quality cuts
print(f"\nQuality cuts: {quality_cuts}")
event_data = get_stereo_events(event_data, quality_cuts)

Now we could apply some other cuts, e.g. excluding the MAGIC-stereo combination events, since they could be poorly reconstructred.

In [ ]:
print("Excluding the MAGIC-stereo combination events...")
event_data.query("combo_type != 0", inplace=True)

### Having a closer look to DL2 data
Let's take a look at the table "event_data", so we understand better how DL2 looks like:

In [ ]:
# Show the data frame
event_data.head()

Each event listed above has parameters from 2 or 3 telescopes. In the step below we compute the means over these DL2 parameters such that each event will be assigned with average parameters for the involved telescopes. Default option of the magicctapipe.io get_dl2_mean function is "simple"(i.e.: no weights)

In [ ]:
print("Calculating mean DL2 parameters...")
event_data_mean = get_dl2_mean(event_data)

# Show the data frame
event_data_mean.head()

Now we use the function `np.diff()` to calculate the time difference between each consecutive event pair (timestamp) then sum up all of them to get the total observation time.

In [ ]:
time_diffs = np.diff(event_data_mean["timestamp"])
obs_time = time_diffs[time_diffs < 1].sum() * u.s

print(f"Observation time: {obs_time.to('min'):.1f}")

The timestamp is given in units of seconds, that is way in the table above it looks like all of the timestamps are the same, but obviously it is just the format they are printed...

In [ ]:
print(event_data_mean["timestamp"].values[0])
print(event_data_mean["timestamp"].values[1])

### Defining ON and OFF regions

In order to observe the target and the background at the same time, a common practice with IACTs is to misspoint the target by a wobble angle (typically 0.4$^{\circ}$ with MAGIC) such that the target and the background points lie in the same "ring" in the camera, as shown in the figure (Archambault et al. 2017):

![wobble](./figures/wobble_Archambault_2017.png)

The region named "ON" is where our target, in this case the Crab Nebula, is. From now on, we will adopt 3 OFF regions that lie in the same camera ring. Let's set the ON-OFF coordinates of our target:

In [ ]:
source_name = "Crab"
n_regions_off = 3

on_coord = SkyCoord.from_name(source_name, frame="icrs")

print(f"ON coordinate ({source_name}):")
print(on_coord)

#Let's also define a size to our sky map:
xlim = on_coord.ra.to_value("deg") + np.array([2, -2])
ylim = on_coord.dec.to_value("deg") + np.array([-2, 2])

print(f"\nNumber of OFF regions: {n_regions_off}")

For each MAGIC-LST run we have a single observation ID and a single wobble. Below we do a loop over few runs and plot the corresponding ON-OFF regions and the center of the camera. 

In [ ]:
# Loop over observation IDs
obs_ids = np.unique(event_data_mean.index.get_level_values("obs_id"))

#nrun_to_show=3
#obs_ids_limited = [x for index, x in enumerate(obs_ids) if index < nrun_to_show]
#for obs_id in obs_ids_limited:

for obs_id in obs_ids:

    print(f"\nObservation ID: {obs_id}")
    df_events = event_data_mean.query(f"obs_id == {obs_id}")

    event_coords = SkyCoord(
        u.Quantity(df_events["reco_ra"], unit="deg"),
        u.Quantity(df_events["reco_dec"], unit="deg"),
        frame="icrs",
    )

    pnt_ra_mean = df_events["pointing_ra"].mean() * u.deg
    pnt_dec_mean = df_events["pointing_dec"].mean() * u.deg

    # Create a figure
    plt.figure(figsize=(8, 8))
    plt.title(f"Observation ID: {obs_id}")
    plt.xlabel("RA [deg]")
    plt.ylabel("Dec [deg]")
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.grid()

    # Plot the ON coordinate
    plt.scatter(
        on_coord.ra.to("deg"), on_coord.dec.to("deg"), label="ON", marker="*", s=400
    )

    # Plot the pointing direction
    plt.scatter(
        pnt_ra_mean, pnt_dec_mean, label="Camera center", marker="o", s=200, color="grey"
    )

    # Calculate the angular distances from the ON region
    theta_on = on_coord.separation(event_coords)
    theta2_on = theta_on.to_value("deg") ** 2

    event_data_mean.loc[(obs_id, slice(None)), "theta2_on"] = theta2_on

    # Calculate the OFF coordinates
    off_coords = calculate_off_coordinates(
        pointing_ra=pnt_ra_mean,
        pointing_dec=pnt_dec_mean,
        on_coord_ra=on_coord.ra,
        on_coord_dec=on_coord.dec,
        n_regions=n_regions_off,
    )

    # Loop over every OFF coordinate
    print("\nOFF coordinates:")

    for i_off, off_coord in off_coords.items():

        print(off_coord)

        # Calculate the angular distance from the OFF coordinate
        theta_off = off_coord.separation(event_coords)
        theta2_off = theta_off.to_value("deg") ** 2

        event_data_mean.loc[(obs_id, slice(None)), f"theta2_off{i_off}"] = theta2_off

        # Plot the OFF coordinate
        plt.scatter(
            off_coord.ra.to("deg"),
            off_coord.dec.to("deg"),
            label=f"OFF{i_off}",
            marker="x",
            s=200,
            color=colors[i_off],
        )

    plt.legend()
    plt.show()

### Applying a rough selection of gammas and event type

We now select only the events seen by all 3 telescopes and with a high  _gammaness_ score.

In [ ]:
# Only events observed with all 3 telescopes:
combo_types = [3] # You can play with different combinations, e.g. with [2,3], or [2] 
# Only events observed with high prob of being gamma rays:
cut_value_gh = 0.8

print(f"Combination types: {combo_types}")
print(f"Global gammaness cut: {cut_value_gh}")

# Get the photon list
event_list = event_data_mean.query(
    f"(combo_type == {combo_types}) & (gammaness > {cut_value_gh})"
).copy()

print(f"\nNumber of events: {len(event_list)}")

# Show the event list
event_list.head()

### Producing a first simple skymap

Let's quickly do a counts map to see what we have selected.

In [ ]:
plt.figure()
plt.title(f"gammaness > {cut_value_gh}, combo_type = {combo_types}")
plt.xlabel("RA [deg]")
plt.ylabel("Dec [deg]")
plt.xlim(xlim)
plt.ylim(ylim)

# Plot the count map
plt.hist2d(
    event_list["reco_ra"],
    event_list["reco_dec"],
    bins=[np.linspace(xlim[1], xlim[0], 101), np.linspace(ylim[0], ylim[1], 101)],
)

plt.colorbar(label="Number of events")
plt.axis(xlim.tolist() + ylim.tolist())
plt.grid()

# Plot the ON coordinate
plt.scatter(
    on_coord.ra.to("deg"),
    on_coord.dec.to("deg"),
    marker="x",
    s=100,
    color="red",
    label="ON",
)

plt.legend()

### Check the $\theta^2$ distributions

$\theta$ is defined as the offset angle between the real position of the target and the reconstructed position for a single event, as shown in the figure below:

![theta](./figures/theta.png)


So let's start by setting up a range at which we want to look for events, let's say from 0 to 0.9$^{\circ}$, and let's also define a cutting value at $\theta = 0.2^{\circ}$ (or $\theta^2 = 0.04^{\circ}$):

In [ ]:
theta2_bins = np.linspace(0, 0.9, 81)  # unit: [deg2]
cut_value_theta2 = 0.04  # unit: [deg2]

theta2_bins_center = (theta2_bins[:-1] + theta2_bins[1:]) / 2

theta2_bins_width = [
    theta2_bins_center - theta2_bins[:-1],
    theta2_bins[1:] - theta2_bins_center,
]

Now let's plot this distribution and compute the significance of the Crab signal.
Since the distribution of photons is not Poissonian, the proper way to go is using the Li & Ma significance (https://arxiv.org/abs/astro-ph/0411660).

In [ ]:
plt.figure()
plt.title(f"gammaness > {cut_value_gh}, combo_type = {combo_types}")
plt.xlabel("Theta2 [deg$^2$]")
plt.ylabel("Number of events")
plt.grid()

# Plot the theta2 distribution from the ON coordinate
hist_on = plt.hist(
    event_list["theta2_on"], bins=theta2_bins, histtype="bar", linewidth=2, alpha=0.5
)[0]

plt.errorbar(
    x=theta2_bins_center,
    y=hist_on,
    xerr=theta2_bins_width,
    yerr=np.sqrt(hist_on),
    fmt="o",
    markersize=5,
    label="ON",
    color=colors[0],
)

# Plot the theta2 distrubutions from the OFF coordinates
for i_off in range(n_regions_off):

    hist_off, _ = np.histogram(event_list[f"theta2_off{i_off+1}"], bins=theta2_bins)

    plt.errorbar(
        x=theta2_bins_center,
        y=hist_off,
        xerr=theta2_bins_width,
        yerr=np.sqrt(hist_off),
        fmt="o",
        markersize=1,
        label=f"OFF{i_off+1}",
    )

# Plot the theta2 cut
print(f"Global theta2 cut: {cut_value_theta2} [deg2]")

plt.plot(
    np.repeat(cut_value_theta2, 2),
    (0, plt.ylim()[1]),
    linestyle="--",
    color="black",
    label=f"Theta2 cut",
)

plt.legend(loc="upper right")

# Calculate the numbers ON, OFF and excess events
n_on = len(event_list.query(f"theta2_on < {cut_value_theta2}"))
print(f"\nNon: {n_on}\n")

n_off_total = 0

for i_off in range(n_regions_off):

    n_off = len(event_list.query(f"theta2_off{i_off+1} < {cut_value_theta2}"))
    print(f"Noff{i_off+1}: {n_off}")

    n_off_total += n_off

print(f"--> Total Noff: {n_off_total}")

alpha = 1 / n_regions_off
n_off_scaled = n_off_total * alpha

print(f"\nalpha = {alpha:.3f}")
print(f"--> Scaled Noff: {n_off_scaled:.0f}")

n_excess = n_on - n_off_scaled
excess_rate = n_excess / obs_time.to_value("min")

print(f"\nNexcess: {n_excess:.0f}")
print(f"Observation time: {obs_time.to('min'):.1f}")
print(f"--> Excess rate: {excess_rate:.1f} [1/min]")

# Calculate the Li&Ma significance
stat = WStatCountsStatistic(n_on, n_off_total, alpha=alpha)
significance = stat.sqrt_ts

print(f"\nLi&Ma significance: {significance:.1f} sigma")

We now repeat the process, but this time slicing the data in different energy bins. We see that the distribution of events gets tighter and the background drops for higher energies.

In [ ]:
# ================
# === Settings ===
# ================

energy_bins = np.logspace(-1, 1, 11)[:-4]

energy_bins_center = (energy_bins[:-1] + energy_bins[1:]) / 2

energy_bins_width = [
    energy_bins[1:] - energy_bins_center,
    energy_bins_center - energy_bins[:-1],
]
# ============
# === Main ===
# ============

n_columns = 3
n_rows = int(np.ceil(len(energy_bins[:-1]) / n_columns))

grid = (n_rows, n_columns)
locs = list(itertools.product(range(n_rows), range(n_columns)))

plt.figure(figsize=(20, n_rows * 8))

# Loop over every energy bin
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):

    plt.subplot2grid(grid, locs[i_bin])
    plt.title(f"{eng_lo:.3f} < energy < {eng_hi:.3f} [TeV]")
    plt.xlabel("Theta2 [deg$^2$]")
    plt.ylabel("Number of events")
    plt.grid()

    # Apply the energy cut
    df_events = event_list.query(
        f"(reco_energy > {eng_lo}) & (reco_energy < {eng_hi})"
    ).copy()

    if len(df_events) > 0:

        # Plot the theta2 distribution from the ON coordinate
        hist_on = plt.hist(
            df_events["theta2_on"],
            bins=theta2_bins,
            histtype="bar",
            linewidth=2,
            alpha=0.5,
        )[0]

        plt.errorbar(
            x=theta2_bins_center,
            y=hist_on,
            xerr=theta2_bins_width,
            yerr=np.sqrt(hist_on),
            fmt="o",
            markersize=5,
            label="ON",
            color=colors[0],
        )

        # Plot the theta2 distribution from the OFF1 coordinate
        hist_off, _ = np.histogram(df_events["theta2_off1"], bins=theta2_bins)

        plt.errorbar(
            x=theta2_bins_center,
            y=hist_off,
            xerr=theta2_bins_width,
            yerr=np.sqrt(hist_off),
            fmt="o",
            markersize=1,
            label=f"OFF1",
        )

        plt.legend(loc="upper right")

### Angular resolution

Finally, let's check the dependency of the angular resolution with the energy of the incoming photons.

In [ ]:
#get event theta value
theta2vec= event_list["theta2_on"]
thetavec=u.Quantity(np.sqrt(theta2vec), u.deg)
#add theta value to dataframe and get a QTable from it
event_list['theta']=thetavec
event_list_table=QTable.from_pandas(event_list)
event_list_table[:5]
#Apply global theta cut to events
mask=event_list_table['theta'].value<np.repeat(np.sqrt(0.04),(len(event_list_table)))
event_list_table=event_list_table[mask]
event_list_table['reco_energy']=u.Quantity(event_list_table['reco_energy'],u.TeV)
#angular resolution
angres_table_Crab = angular_resolution(event_list_table, u.Quantity(energy_bins,u.TeV), energy_type="reco"
)

angres_Crab = angres_table_Crab["angular_resolution"].value
print(f"Angular resolution:\n{angres_Crab} deg")
print(f"Energy bin center:\n{energy_bins_center} TeV")

In [ ]:
plt.figure()

plt.title(f"Crab data angular resolution")
plt.ylabel("Angular resolution (68% cont.) [deg]")
plt.xlabel("Energy [TeV]")
plt.semilogx()
plt.grid()


plt.errorbar(
    x=energy_bins_center,
    y=angres_Crab,
    xerr=energy_bins_width,
    label="Crab",
    marker="o",
)
